Для первой задачи мы используем данные [Jester Online Joke Recommender System](https://goldberg.berkeley.edu/jester-data/)

**Описание данных**

Файл `train_joke_df.csv` содержит:
- UID - id пользователей
- JID - id шуток, которые 
- Ratin - рейтинг шутки, который проставил пользователь 


Рейтинг имеет значение от -10.00 до 10.00. Могут встречаться значения 99.00, но это обозначает Null (нет рейтинга от пользователя).

Метрика для оценки [RMSE](https://www.codecamp.ru/blog/how-to-interpret-rmse/)

Минимальный RMSE: `4.2238`



In [1]:
#!pip install "scikit-surprise==1.1.3"

In [2]:
#!pip install "xlrd==2.0.1"

### Import

In [3]:
import numpy as np
import pandas as pd
from collections import defaultdict
from surprise import Dataset, Reader, CoClustering, accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split as tts
from surprise.model_selection import KFold

np.random.seed(42)

### Базовые функции для скоринга и получения рекомендаций

In [4]:
def get_num_user_ratings(uid):
    """ возвращает кол-во рейтингов у пользователя 
    args: 
      uid: id пользователей
    returns: 
      кол-во объектов, которые оценил пользователь
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # пользователя не было во время обучения (новый, отправить на стартовые рекомендации)
        return 0
    
def get_num_item_ratings(iid):
    """ возвращает кол-во пользователей, которые оценили выбранный элемент 
    args:
      iid: строка с элементов рекомендации
    returns:
      кол-во пользователей, которые дали оценки по элементу
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
# На основе Surprise FAQ построим рекомендации Топ-N
def get_top_n(predictions, n=5):
    """Определят Топ-N рекомендаций

    Args:
        predictions(list of Prediction objects): Списко рекомендаций, из алгоритма Surprise
        n(int): Кол-во топ рекомендаций

    Returns:
        Словарь пользователь - список рекомендакиций для пользователей
        [(raw item id, rating estimation), ...]
    """

    # Предикт для каждого пользователя
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Сортировка предикта (по пользователям)
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n    

### Загрузка и обработка данных

In [5]:
df = pd.read_csv(r'data\recsys-in-practice\train_joke_df.csv')

df.head(5)

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60


In [6]:
# сделаем сортировку и перепишем index
df = df.sort_values(by=['UID', 'JID'])
df = df.reset_index(drop=True)

In [7]:
# создадим на основе набора данных
# поднабор, который требуется для библиотеки Surprise

# указываем минимальный и максимальный рейтинги
reader = Reader(rating_scale=(-10, 10))

# передаём набор, указывая последовательность колонок: user (raw) ids, item (raw) ids, ratings
# для Surprise - это обязательно
data = Dataset.load_from_df(df[['UID', 'JID', 'Rating']], reader)

In [8]:
trainset_data = data.build_full_trainset()

# сделаем разделение на обучающую и тестовую выборку
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

### Обучение модели

In [12]:
# определим набор данных для GridSearchCV
sim_options = {
    "n_cltr_u": [3, 10, 20, 50, 100, 1000, 5000, 15000, 20000, 21000, 22000, 23000],
    "n_cltr_i": [5, 10, 20, 50],
}

param_grid = {"n_cltr_u": [3, 10, 20, 50, 100, 1000, 5000, 15000, 20000, 21000, 22000, 23000],
    "n_cltr_i": [5, 10, 20, 50], }

gs = GridSearchCV(CoClustering, param_grid, measures=["rmse", "mae"], cv=2,)
gs.fit(data)
     
# результат
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

KeyboardInterrupt: 

In [ ]:
# обучим с лучшими параметрами
algo = gs.best_estimator['rmse']
algo.fit(trainset)

# получим предикт и посмотрим метрику
predictions = algo.test(testset)
accuracy.rmse(predictions)

### Тестирование и результаты

In [ ]:
# получаем предикт
uid = 1  # id пользователя 
iid = 1  # iв шутки

# получим предик на основе обученных данных
# -7.82 - это фактический рейтинг, но посмотрим, какой ответ будет в предикте
pred = algo.predict(uid, iid, r_ui=-7.82, verbose=True)

In [ ]:
uid = 24983  # id пользователя 
iid = 62     # iв шутки

pred = algo.predict(uid, iid, r_ui=-0.29, verbose=True)

### Обзор рекомендаций

In [ ]:
# построим таблицу для обзора набора рекомендаций
# посмотрим, какие элементы и в каком кол-ве рекомендуем
trainset = algo.trainset

predictions_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])

predictions_df['№ кол-во пользовательских рейтингов'] = predictions_df.uid.apply(get_num_user_ratings)
predictions_df['№ кол-во рейтингов элементов'] = predictions_df.iid.apply(get_num_item_ratings)
predictions_df['error'] = abs(predictions_df.est - predictions_df.rui)

best_predictions = predictions_df.sort_values(by='error')[:10]
worst_predictions = predictions_df.sort_values(by='error')[-10:]

In [ ]:
best_predictions.head(5)

In [ ]:
# Предикт для всех, кого нет в выборке для обучения
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions)

# Сделаем вывод рекомендаций
a=0
for uid, user_ratings in top_n.items():
    a+=1
    print(uid, [iid for (iid, _) in user_ratings])
    
    if a==10:
        break

### Для отправки на тестирование

In [ ]:
test = pd.read_csv(r'data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)
test.head(5)

In [ ]:
test['Rating'] = test[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                      axis = 1)
                                                      


In [ ]:
# вид набора данных, который должен быть отправлен для тестирования
test['Rating'].to_frame().head(5)

In [ ]:
# формирование файла для отправки в Kaggle
test['Rating'].to_frame().to_csv('baseline.csv')

In [ ]:
df_user_group = df.groupby('UID').agg({'Rating':'mean'}).rename(columns={'Rating':'Rating_mean'})
df_user_group

In [ ]:
df_joke_group = df.groupby('JID').agg({'Rating':'mean'}).rename(columns={'Rating':'Rating_mean'})
df_joke_group

In [ ]:
df_user_group[np.abs(df_user_group['Rating_mean'])> 7]

In [ ]:
test = test.join(df_user_group, on='UID')
test

In [ ]:
test[test['UID'] == 5]

In [ ]:
test = test.rename(columns={'Rating':'Rating_old'})

In [ ]:
test['Rating'] = (test['Rating_old'] + test['Rating_mean']) / 2

In [ ]:
test

In [ ]:
# вид набора данных, который должен быть отправлен для тестирования
test['Rating'].to_frame().head(5)

In [ ]:
# формирование файла для отправки в Kaggle
test['Rating'].to_frame().to_csv('baseline_mean.csv')

In [ ]:
algo.predict(11228, 39, verbose=False)

In [ ]:
algo.estimate(u=11228, i=39)

In [ ]:
algo.sim.shape